In [ ]:
# !pip install rasterio
!pip install geopandas

In [ ]:
from rasterio.mask import mask
import rasterio
import geopandas as gpd

In [ ]:
def cropraster(raster,geometry):
    maska,transform=rasterio.mask.mask(raster, geometry, crop=True,nodata=0,all_touched=True,filled=True)
    image=rasterio.features.rasterize(geometry, out_shape=maska.shape[1:],fill=0,all_touched=True,transform=transform)
    len_ras=image.sum()
    return maska,transform,len_ras

In [17]:
input_raster_path="D:/dem file ku/clippedcatchmentdem.tif"
output_mask_path="D:/dem file ku/output_mask.tif"
input_shp_path="D:/dem file ku/catchment_gandaki_arughat.shp"
gdf = gpd.read_file(input_shp_path)
raster_img=rasterio.open(input_raster_path)
aoi_geometry = gdf.geometry.iloc[0]

In [19]:
maska,transform,len_ras=cropraster(raster_img,[aoi_geometry])

In [26]:
# Set up rasterio metadata
height, width = raster_img.shape
count = 1  # Number of bands
dtype = raster_img.read ().dtype

# Define rasterio profile
profile = {
    'driver': 'GTiff',
    'height': height,
    'width': width,
    'count': count,
    'dtype': dtype,
    'transform': transform
}

# Write ndarray to GeoTIFF file
with rasterio.open(output_mask_path, 'w', **profile) as dst:
    dst.write(maska)  # Assuming single band data